## CRAWL dari KOMPAS

### Def untuk crawling dari Portal KOMPAS
Note : 
- ganti User-Agent pada headers -> ke user agent browser sendiri

In [1]:
import os
import time
import csv
import requests
from bs4 import BeautifulSoup

def kompas_batch(keywords, n_berita=200):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
    }

    save_dir = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/corpus/kompas"
    os.makedirs(save_dir, exist_ok=True)

    for keyword in keywords:
        print(f"/n🚀 Mulai scraping: {keyword}")
        data_kompas = []
        page = 1

        while len(data_kompas) < n_berita:
            alamatURL = f'https://search.kompas.com/search?q={keyword}&sortby=time&page={page}'
            req = requests.get(alamatURL, headers=headers)
            print(f"🔄 Halaman {page} untuk {keyword}")

            if req.status_code != 200:
                print("❌ Gagal akses halaman.")
                break

            # pakai parser fallback kalau lxml error
            try:
                soup = BeautifulSoup(req.content, 'lxml')
            except Exception:
                soup = BeautifulSoup(req.content, 'html.parser')

            articles = soup.find_all('div', class_='articleItem')
            if not articles:
                print("❌ Tidak ada artikel lagi.")
                break

            for article in articles:
                try:
                    a_tag = article.find('a', href=True)
                    if not a_tag:
                        continue

                    url = a_tag['href']
                    title_tag = article.find('h2', class_='articleTitle')
                    judul = title_tag.get_text(strip=True) if title_tag else a_tag.get_text(strip=True)

                    # buka halaman detail
                    detail_req = requests.get(url, headers=headers)
                    try:
                        detail_soup = BeautifulSoup(detail_req.content, 'lxml')
                    except Exception:
                        detail_soup = BeautifulSoup(detail_req.content, 'html.parser')

                    body = detail_soup.find('div', class_='read__content')
                    berita = body.get_text(separator=' ', strip=True) if body else "Tidak ditemukan"

                    if berita and len(berita) > 30:
                        data_kompas.append({
                            'judul': judul,
                            'berita': berita,
                            'url': url,
                        })
                        print(f"[{len(data_kompas)}] {judul}")

                        if len(data_kompas) >= n_berita:
                            break

                    time.sleep(1)  # delay biar ga ke-block

                except Exception as e:
                    print("⚠ Error ambil artikel:", e)
                    continue

            page += 1

        # Simpan hasil per keyword
        if data_kompas:
            filename = os.path.join(save_dir, f"kompas_news_{keyword.lower().replace(' ','')}_{time.strftime('%Y%m%d_%H%M%S')}.csv")
            with open(filename, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader()
                writer.writerows(data_kompas)

            print(f"/n✅ {len(data_kompas)} artikel '{keyword}' disimpan ke {filename}")
        else:
            print(f"❌ Tidak ada data valid untuk '{keyword}'")



In [ ]:
keywords = [
    "BI rate",
    "Pertumbuhan ekonomi",
    "Inflasi",
    "Bank Indonesia",
    "Perbankan digital",
    "IHSG",
    "Pelemahan Rupiah",
    "Investasi",
    "Ekonomi Indonesia",
    "Danantara",
    "OJK",
    "Reksadana",
    "Bursa Efek Indonesia",
    "BEI",
    "Ekonomi digital"
]

kompas_batch(keywords, n_berita=300)


/n🚀 Mulai scraping: BI rate


🔄 Halaman 1 untuk BI rate
[1] BI Rate Turun, OJK Imbau Perbankan Sesuaikan Tingkat Suku Bunga
[2] Prospek Saham BBCA di Tengah Penurunan BI Rate
[3] Penempatan Dana Pemerintah Rp 200 T dan Pemangkasan BI Rate Jadi Bahan Bakar Baru Pertumbuhan Ekonomi
[4] Ekonom Prediksi BI Rate Bertahan di 5 Persen Pada September 2025, Ini Alasannya
[5] Rp 200 T Mengalir, BI Rate 4,75 Persen: Di Persimpangan Rupiah dan Kredit Murah
[6] Bank Indonesia Turunkan Suku Bunga Acuan untuk Kelima Kalinya, BI Rate Kini 4,75 Persen
[7] Rate Halte TJ yang Paling Bikin Ngos-Ngosan
[8] Kendala Pemasangan Lampu Bi-LED dan Solusinya
[9] BI: Pertumbuhan Ekonomi Indonesia Perlu Semakin Ditingkatkan
[10] Kenapa Rupiah Melemah? Ini Penjelasan Bos BI
[11] Bos BI Jelaskan Penyebab Rupiah Melemah
[12] Cara Mudah Memperbaiki Skor BI Checking, Resmi OJK
[13] Candaan Purbaya Saat Jumpa Bos BI, Diplomasi Bebek Goreng
[14] Tips Memilih Lampu Bi-LED yang Cocok untuk Mobil Harian
[15] Biaya Modifikasi Lampu Bi-LED untuk Mobil Jepa